In [ ]:
!pip install scapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scapy: filename=scapy-2.5.0-py2.py3-none-any.whl size=1444328 sha256=eeec4ce2be7b67fad792b1bccbd582e6986c3182302682c97ff87af9fd039110
  Stored in directory: /root/.cache/pip/wheels/82/b7/03/8344d8cf6695624746311bc0d389e9d05535ca83c35f90241d
Successfully built scapy


In [ ]:
!pip install tensorflow


 Preprocessing

In [ ]:
from scapy.all import *
import pandas as pd
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam


In [ ]:
# Extracting the features out of the real Darkgate file
def extract_features(pcap_file):
    packets = rdpcap(pcap_file)
    features = []

    for i, packet in enumerate(packets):
        if i > 0:
            # Calculate inter-arrival time
            inter_arrival_time = packets[i].time - packets[i-1].time
        else:
            inter_arrival_time = 0

        if IP in packet:
            ip_src = packet[IP].src
            ip_dst = packet[IP].dst
        else:
            ip_src = ip_dst = None

        if TCP in packet:
            protocol = 'TCP'
            src_port = packet[TCP].sport
            dst_port = packet[TCP].dport
        elif UDP in packet:
            protocol = 'UDP'
            src_port = packet[UDP].sport
            dst_port = packet[UDP].dport
        else:
            protocol = src_port = dst_port = None

        packet_size = len(packet)

        features.append([ip_src, ip_dst, src_port, dst_port, protocol, packet_size, inter_arrival_time])

    df = pd.DataFrame(features, columns=['IP Source', 'IP Destination', 'Source Port', 'Destination Port', 'Protocol', 'Packet Size', 'Inter-Arrival Time'])
    return df
pcap_file = 'Darkgate1.pcap'
df = extract_features(pcap_file)
# Drop the IP address columns from the DataFrame
df_processed = df.drop(columns=['IP Source', 'IP Destination'])


#csv_file = pcap_file.replace('.pcap', '.csv')
#df.to_csv(csv_file, index=False)
#print(f'Features extracted and saved to {csv_file}')

Normalize numerical features

In [ ]:
scaler = MinMaxScaler()
numerical_features = ['Packet Size', 'Inter-Arrival Time']
df_processed[numerical_features] = scaler.fit_transform(df_processed[numerical_features])

Encode categorical features

In [ ]:
df_processed = pd.get_dummies(df_processed, columns=['Protocol'])

KeyError: "None of [Index(['Protocol'], dtype='object')] are in the [columns]"

Split the dataset

In [ ]:
X_train = df_processed.values.astype(np.float32)

ValueError: could not convert string to float: '10.127.0.135'

Define the generator

In [ ]:
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(X_train.shape[1], activation='tanh'))
    return model

Define the discriminator

In [ ]:
def build_discriminator():
    model = Sequential()
    model.add(Dense(256, input_dim=X_train.shape[1]))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    return model

Build the GAN

In [ ]:
def build_gan(generator, discriminator):
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

Set hyperparameters

In [ ]:
latent_dim = 100
epochs = 100
batch_size = 32

Build and compile the models

In [ ]:
generator = build_generator(latent_dim)
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])

gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))


Train the GAN

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Preprocess the data
X_train_processed = X_train.copy()

# Convert string columns to numeric using LabelEncoder
string_columns = ['IP Source', 'IP Destination']  # Replace with your string column names
label_encoders = {}
# Apply LabelEncoder to each string column
for column in string_columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# After preprocessing (e.g., scaling or one-hot encoding), convert the DataFrame to a NumPy array
X_train_processed = df.values.astype(np.float32)

for epoch in range(epochs):
    # Generate fake samples
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_samples = generator.predict(noise)

    # Get real samples
    real_samples = X_train_processed[np.random.randint(0, X_train_processed.shape[0], batch_size)]

    # Train the discriminator
    X = np.concatenate((real_samples, fake_samples))
    y_dis = np.zeros(2 * batch_size, dtype=np.float32)
    y_dis[:batch_size] = 0.9  # Label real samples as 0.9
    discriminator.trainable = True

    discriminator.train_on_batch(X, y_dis)

    # Train the generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    y_gen = np.ones(batch_size, dtype=np.float32)  # Label fake samples as 1
    discriminator.trainable = False
    gan.train_on_batch(noise, y_gen)

1/1 [==============================] - 0s 18ms/step


Evaluate the GAN

In [ ]:
noise = np.random.normal(0, 1, (batch_size, latent_dim))
generated_samples = generator.predict(noise)

# Predict using the discriminator
predictions = discriminator.predict(X_train.astype(np.float32))

# Classify samples as real or fake based on a threshold
threshold = 0.5
predicted_labels = (predictions > threshold).astype(int)

# Assuming you have the true labels for the Darkgate1 dataset
true_labels = ...  # Replace with your true labels

# Calculate accuracy
accuracy = np.mean(predicted_labels == true_labels)
print(f"Accuracy: {accuracy}")

1/1 [==============================] - 0s 18ms/step


ValueError: could not convert string to float: '10.127.0.135'